# Englsih Premier League 2018/2019 seasons W/D/L prediction

In [27]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

%matplotlib inline

In [102]:
df = pd.read_csv(r'D:\git_space\data\Premier-League_18-19seasons_matches.csv')
df.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1533927600,Aug 10 2018 - 7:00pm,complete,74439,Manchester United,Leicester City,Andre Marriner,1,0.0,0.0,...,1.37,4.98,9.81,1.33,2.00,3.60,7.50,2.05,1.69,Old Trafford (Manchester)
1,1533987000,Aug 11 2018 - 11:30am,complete,51749,Newcastle United,Tottenham Hotspur,Martin Atkinson,1,0.0,0.0,...,4.51,3.77,1.84,1.31,1.95,3.50,5.75,1.83,1.87,St. James' Park (Newcastle upon Tyne)
2,1533996000,Aug 11 2018 - 2:00pm,complete,10353,AFC Bournemouth,Cardiff City,Kevin Friend,1,0.0,0.0,...,2.03,3.51,3.96,1.31,1.95,3.45,6.70,1.83,1.83,Vitality Stadium (Bournemouth- Dorset)
3,1533996000,Aug 11 2018 - 2:00pm,complete,24821,Fulham,Crystal Palace,Mike Dean,1,0.0,0.0,...,2.31,3.44,3.26,1.28,1.87,3.20,7.00,1.71,2.00,Craven Cottage (London)
4,1533996000,Aug 11 2018 - 2:00pm,complete,24121,Huddersfield Town,Chelsea,Chris Kavanagh,1,0.0,0.0,...,7.47,4.27,1.51,1.29,1.91,3.30,6.95,2.00,1.71,John Smith's Stadium (Huddersfield- West Yorks...


### PPG: Points Per Game, Pre-match는 프리시즌이기 때문에 쓸모없다 FT(Full Time) odds: 풀타임 배댱률 xg: Expected Goals

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 64 columns):
timestamp                              380 non-null int64
date_GMT                               380 non-null object
status                                 380 non-null object
attendance                             380 non-null int64
home_team_name                         380 non-null object
away_team_name                         380 non-null object
referee                                380 non-null object
Game Week                              380 non-null int64
Pre-Match PPG (Home)                   380 non-null float64
Pre-Match PPG (Away)                   380 non-null float64
home_ppg                               380 non-null float64
away_ppg                               380 non-null float64
home_team_goal_count                   380 non-null int64
away_team_goal_count                   380 non-null int64
total_goal_count                       380 non-null int64
total_goals_

In [104]:
df.isnull().sum()

timestamp         0
date_GMT          0
status            0
attendance        0
home_team_name    0
                 ..
odds_ft_over35    0
odds_ft_over45    0
odds_btts_yes     0
odds_btts_no      0
stadium_name      0
Length: 64, dtype: int64

In [105]:
# 의미없는 column 삭제
meaningless_data = ['timestamp', 'date_GMT', 'status', 'referee', 'Game Week', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)',                          'average_goals_per_match_pre_match', 'btts_percentage_pre_match', 'over_15_percentage_pre_match', 'over_25_percentage_pre_match', 'over_35_percentage_pre_match', 'over_45_percentage_pre_match', 'over_15_HT_FHG_percentage_pre_match', 'over_05_HT_FHG_percentage_pre_match', 'over_15_2HG_percentage_pre_match', 'over_05_2HG_percentage_pre_match', 'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match', 'odds_ft_over15', 'odds_ft_over25', 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes', 'odds_btts_no', 'stadium_name']

# 경기시작 전에 알 수 없는 column 삭제(예측을 해야 하기 때문에 경기 기록 data를 사용하면 안된다)
unknown_data = ['total_goal_count', 'total_goals_at_half_time', 'home_team_goal_count_half_time', 'away_team_goal_count_half_time', 'home_team_goal_timings', 'away_team_goal_timings', 'home_team_corner_count', 'away_team_corner_count', 'home_team_yellow_cards', 'home_team_red_cards', 'away_team_yellow_cards', 'away_team_red_cards', 'home_team_first_half_cards', 'home_team_second_half_cards', 'away_team_first_half_cards', 'away_team_second_half_cards', 'home_team_shots', 'away_team_shots', 'home_team_shots_on_target', 'away_team_shots_on_target', 'home_team_shots_off_target', 'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls', 'home_team_possession', 'away_team_possession',]

df = df.drop(meaningless_data + unknown_data, axis=1)

df.head()

,attendance,home_team_name,away_team_name,home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win
0,74439,Manchester United,Leicester City,1.89,1.32,2,1,1.25,1.46,1.37,4.98,9.81
1,51749,Newcastle United,Tottenham Hotspur,1.32,1.74,1,2,1.18,1.58,4.51,3.77,1.84
2,10353,AFC Bournemouth,Cardiff City,1.53,0.74,2,0,1.50,0.89,2.03,3.51,3.96
3,24821,Fulham,Crystal Palace,1.11,1.53,0,2,1.67,1.81,2.31,3.44,3.26
4,24121,Huddersfield Town,Chelsea,0.47,1.58,0,3,0.90,1.17,7.47,4.27,1.51


### 팀명은 아직 새롭게 추가할 data를 위하여 남겨놓자

### 현재 data는 승무패가 아닌 점수로 column이 되어있으므로 승무패 column으로 바꿔준다. 이는 home team 기준 승무패로 설정 => 승:2 무:1 패:0
    - away team은 이 반대로 해주고 무만 같게 한다

In [106]:
# 홈팀 득점 수 - 어웨이팀 득점 수 column 생성
df['difference'] = df['home_team_goal_count'] - df['away_team_goal_count']
df.head()

# difference column의 data가 양수(홈팀 승)면 2반환, 0(무승부)이면 1반환, 음수(홈팀 패)면 0반환
def func(x):
    if x > 0:
        return 2
    elif x == 0:
        return 1
    else:
        return 0

# 홈팀의 승무패 정보를 담는 column 생성
df['home_team_result'] = df['difference'].apply(lambda x : func(x))
df

,attendance,home_team_name,away_team_name,home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,difference,home_team_result
0,74439,Manchester United,Leicester City,1.89,1.32,2,1,1.25,1.46,1.37,4.98,9.81,1,2
1,51749,Newcastle United,Tottenham Hotspur,1.32,1.74,1,2,1.18,1.58,4.51,3.77,1.84,-1,0
2,10353,AFC Bournemouth,Cardiff City,1.53,0.74,2,0,1.50,0.89,2.03,3.51,3.96,2,2
3,24821,Fulham,Crystal Palace,1.11,1.53,0,2,1.67,1.81,2.31,3.44,3.26,-2,0
4,24121,Huddersfield Town,Chelsea,0.47,1.58,0,3,0.90,1.17,7.47,4.27,1.51,-3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,53331,Liverpool,Wolverhampton Wanderers,2.79,1.21,2,0,1.40,0.92,1.32,5.80,9.00,2,2
376,74457,Manchester United,Cardiff City,1.89,0.74,0,2,2.67,1.31,1.28,6.20,10.00,-2,0
377,30367,Southampton,Huddersfield Town,1.21,0.37,1,1,1.15,1.04,1.41,4.90,7.50,0,1
378,60124,Tottenham Hotspur,Everton,2.00,1.05,2,2,1.23,1.80,2.10,3.50,3.55,0,1


In [107]:
# 팀의 정보를 하나의 column에 담았으니 쓸모 없어진 홈팀, 어웨이팀 득점 수, difference column은 삭제한다

df.drop(['home_team_goal_count', 'away_team_goal_count', 'difference'], axis=1, inplace=True)
df

,attendance,home_team_name,away_team_name,home_ppg,away_ppg,team_a_xg,team_b_xg,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,home_team_result
0,74439,Manchester United,Leicester City,1.89,1.32,1.25,1.46,1.37,4.98,9.81,2
1,51749,Newcastle United,Tottenham Hotspur,1.32,1.74,1.18,1.58,4.51,3.77,1.84,0
2,10353,AFC Bournemouth,Cardiff City,1.53,0.74,1.50,0.89,2.03,3.51,3.96,2
3,24821,Fulham,Crystal Palace,1.11,1.53,1.67,1.81,2.31,3.44,3.26,0
4,24121,Huddersfield Town,Chelsea,0.47,1.58,0.90,1.17,7.47,4.27,1.51,0
...,...,...,...,...,...,...,...,...,...,...,...
375,53331,Liverpool,Wolverhampton Wanderers,2.79,1.21,1.40,0.92,1.32,5.80,9.00,2
376,74457,Manchester United,Cardiff City,1.89,0.74,2.67,1.31,1.28,6.20,10.00,0
377,30367,Southampton,Huddersfield Town,1.21,0.37,1.15,1.04,1.41,4.90,7.50,1
378,60124,Tottenham Hotspur,Everton,2.00,1.05,1.23,1.80,2.10,3.50,3.55,1
